In [2]:
import pandas as pd
import numpy as np
import time

# Importing dataset

In [3]:
graph = pd.read_csv(
    "data/opsahl-ucsocial/out.opsahl-ucsocial", 
    names=["from", "to", "weight", "timestamp"],
    sep=" |\t",
    skiprows=[0, 1]
)
print(graph.head())

   from  to  weight   timestamp
0     1   2       1  1082008561
1     3   4       1  1082123439
2     5   2       1  1082381991
3     6   7       1  1082407219
4     8   7       1  1082407356


/var/folders/wc/28hh_w_d5zn4rvtk1bw0_g700000gn/T/ipykernel_9227/3009724296.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  graph = pd.read_csv(


In [4]:
V = np.unique(graph["from"]._append(graph["to"]))
n = V.size
volume = graph["timestamp"].size
print(f"{n = }, {volume = }")

n = 1899, volume = 59835


# Chapter 1

## Preparing static graph

In [5]:
graph_static = [set() for _ in range(n +1)]
for row_number, row in graph.iterrows():
    graph_static[row["from"]].add(row["to"])
    graph_static[row["to"]].add(row["from"])


## Task 1.1

In [6]:
E_count = 0
for i in range(len(graph_static)):
    E_count += len(graph_static[i])
E_count //= 2
print(E_count)

p = E_count * 2 / (n * (n - 1))
print(p)

13838
0.007678601848568738


In [32]:
V_to_visit = set(V)
connectivity_components = []
while(V_to_visit):
    V_visited = set()
    queue = []
    for u in V_to_visit:
        queue.append(u)
        break
    while queue:
        u = queue.pop()
        V_to_visit.discard(u)
        V_visited.add(u)
        u_adjacent_to_visit = graph_static[u].difference(V_visited)
        for v in u_adjacent_to_visit:
            if v not in queue:
                queue.append(v)
    connectivity_components.append(V_visited)

sizes = list(map(lambda x: len(x), connectivity_components))
max_val = max(sizes)
max_connectivity_component_index = sizes.index(max_val)
proportion = max_val / len(V)
print(f"{max_val = }, {max_connectivity_component_index = }, {proportion = }")

       
    


max_val = 1893, max_connectivity_component_index = 0, proportion = 0.9968404423380727


In [8]:
# 1.1
print("|V| = %i, |E| = %i, p = %f, number of components = %i, max component proportion = %f" 
      % (n, E_count, p, len(connectivity_components), proportion))

|V| = 1899, |E| = 13838, p = 0.007679, number of components = 4, max component proportion = 0.996840


## Task 1.2

In [37]:
component = list(connectivity_components[max_connectivity_component_index])
# infinite distance = n + 1
distance_matrix = [[n + 1 for j in range(n + 1)] for i in range(n + 1)]
eccentricities = dict()
for start in component:
    V_visited = set()
    queue = [(start, 0)]
    queued = set([start])
    max_depth = 0
    while queue:
        u, depth = queue.pop(0)
        max_depth = max(max_depth, depth)
        queued.discard(u)
        V_visited.add(u)
        u_adjacent_to_visit = graph_static[u].difference(V_visited)
        for v in u_adjacent_to_visit:
            if v not in queued:
                distance = distance_matrix[start][v]
                if depth + 1 < distance:
                    distance_matrix[start][v] = depth + 1
                    distance_matrix[v][start] = depth + 1
                queue.append((v, depth + 1))
                queued.add(v)
    eccentricities[start] = max_depth

In [50]:
component = set(connectivity_components[max_connectivity_component_index])

diameter = max(eccentricities.values())
radius = min(eccentricities.values())

all_distances = []
for i in range(1, n + 1):
    for j in range(i + 1, n + 1):
        if i in component and j in component:
            all_distances.append(distance_matrix[i][j])
all_distances.sort()

In [54]:
ind = 0.9 * len(all_distances) 
print(max(all_distances))
print(ind)
print(all_distances[int(ind)])

8
1611700.2
4


In [57]:
import random
component = list(connectivity_components[max_connectivity_component_index])
random_500_verticles = random.choices(component, k=500)
random_1000_verticles = random.choices(component, k=1000)

random_500_distances = []
random_1000_distances = []

def get_random_distances(verticles, distances):
    for i in range(len(verticles)):
        for j in range(i + 1, len(verticles)):
            u = verticles[i]
            v = verticles[j]
            distances.append(distance_matrix[u][v])
    return distances

random_500_distances = get_random_distances(random_500_verticles, random_500_distances)    
random_1000_distances = get_random_distances(random_1000_verticles, random_1000_distances)  

print(random_500_distances)
       
                    

[3, 3, 3, 4, 4, 4, 2, 3, 3, 3, 2, 3, 4, 3, 3, 3, 3, 4, 3, 2, 4, 3, 3, 2, 3, 4, 3, 2, 3, 3, 4, 4, 3, 4, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 4, 3, 2, 3, 3, 2, 3, 3, 3, 3, 4, 4, 2, 3, 4, 5, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 2, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 4, 2, 3, 4, 3, 3, 3, 3, 4, 3, 2, 2, 3, 3, 4, 3, 3, 3, 2, 3, 3, 3, 4, 3, 3, 2, 3, 4, 3, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3, 4, 3, 2, 4, 3, 3, 2, 2, 2, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 3, 4, 3, 3, 3, 2, 2, 5, 4, 3, 3, 3, 4, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 2, 3, 3, 3, 4, 4, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 2, 2, 4, 2, 4, 2, 3, 3, 3, 3, 2, 4, 3, 3, 3, 2, 3, 2, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 2, 2, 3, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 1, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 2, 4, 3, 3, 3, 2, 2, 2, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 2, 3, 3, 2, 3, 2, 4, 2, 3, 3, 

In [58]:
# 1.2
print("diameter = %i, raduis = %i" 
      % (diameter, radius))

diameter = 8, raduis = 4


## Task 1.3

In [44]:
component = list(connectivity_components[max_connectivity_component_index])

Cl = dict()
for u in component:
    u_neighbors = graph_static[u]

    if len(u_neighbors) < 2:
        Cl[u] = 0
        continue

    Lu_doubled = 0
    for neighbor in u_neighbors:
        Lu_doubled += len(graph_static[neighbor].intersection(u_neighbors))
    Cl[u] = Lu_doubled / (len(u_neighbors) * (len(u_neighbors) - 1))

Cl_average = sum(Cl.values()) / len(Cl.values())

In [45]:
#1.3
print("Cl_average = %f" % (Cl_average))

Cl_average = 0.109746


## Task 1.4

In [51]:
R1 = 0
R2 = 0
R3 = 0
Re = 0
for i in range(1, n + 1):
    ki = len(graph_static[i])
    R1 += ki
    R2 += ki**2
    R3 += ki**3
    for j in range(1, n + 1):
        if j in graph_static[i]:
            kj = len(graph_static[j])
            Re += ki * kj
degree_associativity = (Re * R1 - R2**2) / (R3 * R1 - R2**2)

In [52]:
print(degree_associativity)

-0.1877757871466803
